<a href="https://colab.research.google.com/github/eyaler/avatars4all/blob/master/facevidblur.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#FaceVidBlur

### Notebook by [Eyal Gruss](https://eyalgruss.com), [@eyaler](twitter.com/eyaler)

Blur faces in video

More notebooks: [github.com/eyaler/avatars4all](https://github.com/eyaler/avatars4all)

Shortcut here: [tfi.la/face](https://tfi.la/blur)

In [ ]:
#@title Setup

import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install git+https://github.com/ytdl-org/youtube-dl
!pip install imageio
!pip install git+https://github.com/1adrianb/face-alignment

In [ ]:
#@title Blur faces in video

video_url = 'https://www.youtube.com/watch?v=SoAKSHcrDGg' #@param {type: 'string'}
face_num = 0 #@param {type: 'integer'}
#@markdown (use face_num = 0 to select the all faces, face_num >= 1 to select a face from faces ordered top to bottom and left to right)
min_conf = .8 #@param {type: 'number'}
expose_eyes_mouth = False #@param {type: 'boolean'}
draw_landmarks = True #@param {type: 'boolean'}
landmarks_hex = '0000FF' #@param {type: 'string'}
start_seconds = 0 #@param {type: 'number'}
duration_seconds = 15 #@param {type: 'number'}
#@markdown (use duration_seconds = 0 for unrestricted duration)
max_width = 0 #@param {type: 'integer'}
#@markdown (use max_width = 0 for unrestricted width)
max_height = 0 #@param {type: 'integer'}
#@markdown (use max_height = 0 for unrestricted height)
output_filename = 'output.mp4' #@param {type: 'string'}

need_dl = True
try:
  if video_url == save_url:
    need_dl = False
except:
  pass
if need_dl:
  !rm -f /content/video.mp4
  !rm -f /content/full_video.mp4
  !youtube-dl -f "bestvideo[ext=mp4][vcodec!*=av01]+bestaudio[ext=m4a]/mp4[vcodec!*=av01]/mp4[vcodec!*=av01]/mp4" "$video_url" --merge-output-format mp4 -o /content/video.mp4
  !cp /content/video.mp4 /content/full_video.mp4

need_fix_duration = True
try:
  if video_url == save_url and start_seconds == save_start_seconds and duration_seconds == save_duration_seconds:
    need_fix_duration = False
except:
  pass
if need_fix_duration:
  if start_seconds or duration_seconds:
    !ffmpeg -y -ss $start_seconds -t $duration_seconds -i /content/full_video.mp4 -f mp4 /content/video.mp4
  else:
    !cp /content/full_video.mp4 video.mp4

save_url = video_url
save_start_seconds = start_seconds
save_duration_seconds = duration_seconds

import imageio.v3 as iio
from IPython.display import Video, clear_output
import face_alignment
import numpy as np
from skimage.draw import ellipse, polygon, polygon_perimeter

fa = face_alignment.FaceAlignment(landmarks_type=1)
fps = iio.immeta('/content/video.mp4')['fps']

!rm -rf /content/in_frames
!mkdir -p /content/in_frames

face_parts = {'eyebrow1': slice(17, 22),
              'eyebrow2': slice(22, 27),
              'nose': slice(27, 31),
              'nostril': slice(31, 36),
              'eye1': slice(36, 42),
              'eye2': slice(42, 48),
              'lips': slice(48, 60),
              'teeth': slice(60, 68)
              }

step = len(landmarks_hex) // 3
landmarks_rgb = [int(landmarks_hex[i: i + step], 16) * 17**(step == 1) for i in range(0, len(landmarks_hex), step)]

for i, im in enumerate(iio.imiter('/content/video.mp4')):
  landmarks, _, bboxes = fa.get_landmarks_from_image(im, return_bboxes=True)
  if bboxes is not None:
    lb = zip(landmarks, bboxes)
    if min_conf:
      lb = [(l, b) for l, b in lb if b[-1] >= min_conf]
    if face_num:
      lb = sorted(lb, key=lambda l, b: (b[1], b[3], b[0], b[2]))[face_num - 1 : face_num]
    orig = im.copy()
    for landmarks, (x0, y0, x1, y1, _) in lb:
      x = (x0+x1) / 2
      y = (y0+y1) / 2
      w = x1 - x0
      h = y1 - y0
      yy, xx = ellipse(y, x, h/2, w/2)
      im[yy, xx] = np.median(orig[yy, xx], axis=0)
      if expose_eyes_mouth:
        for name, part in face_parts.items():
          if name in ('eye1', 'eye2', 'lips'):
            yy, xx = polygon(*list(zip(*landmarks[part]))[::-1])
            im[yy, xx] = orig[yy, xx]
    if draw_landmarks:
      for landmarks, _ in lb:
        for part in face_parts.values():
          yy, xx = polygon_perimeter(*list(zip(*landmarks[part]))[::-1])
          im[yy, xx] = landmarks_rgb
  iio.imwrite(f'/content/in_frames/frame_{i:06d}.png', im, compress_level=1)

!ffmpeg -y -framerate $fps -thread_queue_size 0 -i /content/in_frames/frame_%06d.png -i /content/video.mp4 -c:v libx264 -c:a aac -map 0:v -map 1:a? -vf "scale=min(iw\,$max_width):min(ih\,$max_height):force_original_aspect_ratio=decrease:force_divisible_by=2" -pix_fmt yuv420p -profile:v baseline -movflags +faststart "/content/$output_filename"

clear_output()

meta = iio.immeta('/content/' + output_filename)
print(f'w={meta["size"][0]} h={meta["size"][1]} t={meta["duration"]}, fps={meta["fps"]}')
Video('/content/' + output_filename, embed=True, html_attributes="autoplay controls loop")

In [ ]:
#@title Download

from google.colab import files
files.download('/content/' + output_filename)